# Multivariate two factorial ANOVA 

In order to detect masks with early emerging signals we can run varaiance analysis ANOVA. This will tell us if there are differences between SSPs, but will not serve to differantiate new observations. The goal is to detect which masks allows for an early seperation between SSPs, this allows for easier development of ML algs.

Here we run two factorial design hereunder
- Mask as factor, with global levels of nomask, seamask and landmask
- SSP as factor, with levels SSP126, SSP245, SSP370 and SSP585¨
Since all levels of each factor occours in combination we have 12 groups and a crossed model design (in difference to a nested model design). 

We run the factors as descriptive variables for observations of tas, pr, txx and rx5day. Since the interaction between these responsvariables is what will allow us to detect SSP seperation early we use multivariate analysis of variance or MANOVA. 

Since we use a yearly cross-sectional approach for classification we run two-factorial MANOVA on each cross section.  

## Preprocessing of data and initial investigation
In order to enable the analysis we need to transform our data from seperate multivariate time series files on the .nc format, to a table for each seperate cross section (year). The format will be:

| # | Mask | SSP | tas_value | pr_value | txx_value | rx5day_value |
| --|---|---|---|---|---|---|
....

This allows to run the analysis directly if each realization gets one row each. While performing this preprocessing some initial investigations will be performed.


### Investigations of time series ensambles.
It is always important to know the big picture of what you are looking at. Therefor we start by presenting the ensambles of each variable under different masks. In order to calculate the ensambles we use the xclim library. 

**Sources:**
- https://xclim.readthedocs.io/en/stable/index.html
    - https://xclim.readthedocs.io/en/stable/notebooks/ensembles.html

#### Import libraries

In [ ]:
from pathlib import Path
from src.preproces import *
import matplotlib.pyplot as plt
%matplotlib inline
from xclim import ensembles

file_handler = Handle_Files()

: 

Support functions for plotting

In [ ]:
def legend_without_duplicate_labels(fig):
    handles, labels = fig.axes[0].get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    fig.legend(*zip(*unique), loc='center left', bbox_to_anchor=(1, 0.5), fontsize=14)

: 

In [ ]:

fig, axs = plt.subplots(4, 3, figsize=(15, 12), sharey='row')#, sharex='col')
fig2, axs2 = plt.subplots(4, 3, figsize=(15, 12), sharey='row')#, sharex='col')

main_data_dir = '/nird/home/johannef/Masterthesis_S23 DataFiles/AnnualGlobalClimatologies'
masks = ['nomask', 'landmasked', 'seamasked']
SSPs = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
variables = ['tas', 'pr', 'rx5dayETCCDI', 'txxETCCDI']

colors = plt.cm.coolwarm(np.linspace(0, 1, len(SSPs)))
color_map = dict(zip(SSPs, colors))

for i, mask in enumerate(masks):
    
    for j, variable in enumerate(variables):
        axs[j, i].set_ylabel(variable)
        axs[j, i].set_title(f'{variable} ({mask})')
        axs2[j, i].set_ylabel(variable)
        axs2[j, i].set_title(f'{variable} ({mask})')

        means = {}
        stds = {}
        for scenario in SSPs:
            data_dir = '/'.join([main_data_dir, mask, variable, scenario])

            ens = ensembles.create_ensemble(Path(data_dir).glob("*.nc"))
            ens_stats = ensembles.ensemble_mean_std_max_min(ens)
            
            ens_mean = ens_stats[f'{variable}_mean']
            ens_std = ens_stats[f'{variable}_stdev']
            axs[j, i].plot(ens_stats.year, ens_mean, 
                           label=scenario,
                           color=color_map[scenario])
            axs[j, i].fill_between(ens_stats.year, ens_mean - ens_std, ens_mean + ens_std, 
                                   color=color_map[scenario],
                                   alpha=0.5)

            means[scenario] = ens_mean
            stds[scenario] = ens_std
        
        mean = np.mean(means.values())

        for scenario in means.keys(): 
            # meanscaled version
            # Kan ikke skalere med gruppegjennomsnittet må benytte gjennomsnittet av alle sspene
            meanscaled_ens_mean = means[scenario] - mean
            meanscaled_ens_std = stds[scenario] - mean
            axs2[j, i].fill_between(ens_stats.year, meanscaled_ens_mean - meanscaled_ens_std, meanscaled_ens_mean + meanscaled_ens_std, 
                                   color=color_map[scenario],
                                   alpha=0.5)
            axs2[j, i].plot(ens_stats.year, meanscaled_ens_mean, 
                            label=scenario,
                            color=color_map[scenario])
            

legend_without_duplicate_labels(fig)
legend_without_duplicate_labels(fig2)
plt.tight_layout()
plt.show()

: 

### Datawrangling for table creation

In [ ]:
def extract_cross_sections(data_dir, ssp_names, center_year, target_mapping, pm=1):
    
    start_year = center_year - pm
    end_year = center_year + pm  

    df = pd.DataFrame(columns=['pr', 'tas', 'target_ssp'], 
                      index=np.arange(len(ssp_names)*40))
    indx = 0

    for ssp_name in ssp_names:
        ssp_dir = os.path.join(data_dir, ssp_name)
        file_list = os.listdir(ssp_dir)

        for file_name in file_list:
            file_path = os.path.join(ssp_dir, file_name)
            ds = xr.open_dataset(file_path)
            
            ds_years = ds.sel(year=slice(start_year, end_year))
            mean_pr = ds_years.pr.mean(dim='year').values
            mean_tas = ds_years.tas.mean(dim='year').values

            df.loc[indx] = [mean_pr, mean_tas, target_mapping[ssp_name]] 
            
            indx += 1         
    
    return df

: 

### Boxplots 
one for every fift years?


## MANOVA

### MANOVA adequacy checking
s. 167


### MANOVA testing

### Post-hoc Linear disciminant analysis (LDA)